In [2]:
import os
import pandas as pd
import json
import random
from deap import base, creator, tools, algorithms

In [4]:
directory_path = 'train'
list_df_ovens = []
list_df_series = []

In [118]:
import json
import random
import pandas as pd
from deap import base, tools, creator, algorithms

def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def analyze_day(day_data, day):
    df_ovens = pd.DataFrame(day_data['ovens'])
    df_series = pd.DataFrame(day_data['series'])

    df_ovens['day'] = day
    df_series['day'] = day

    print("Средняя начальная температура печей:", df_ovens['start_temp'].mean())
    print("Минимальная температура серий:", df_series['temperature'].min())

    return df_ovens, df_series

def individual_from_series_operations(df_series, df_ovens):
    exploded_series = df_series['operations'].explode()
    unique_operations = set(json.dumps(d, sort_keys=True) for d in exploded_series)

    # Преобразовать словари в строки JSON
    unique_operations_as_strings = [json.dumps(d, sort_keys=True) for d in unique_operations]

    # Получить уникальные строки
    all_unique_operations = list(set(unique_operations_as_strings))

    # Создать отображение из строки в индекс
    operation_indices = {op_str: idx for idx, op_str in enumerate(all_unique_operations)}

    unique_ovens = tuple(df_ovens.index)
    ind_size = len(all_unique_operations) * len(unique_ovens)

    # Вернуть объект индивида
    ind = creator.Individual()
    ind.genotype = [random.choice(list(operation_indices.values())) for _ in range(ind_size)]
    ind.operation_indices = operation_indices
    ind.unique_ovens = unique_ovens

    return ind


def evaluate(ind, df_ovens, df_series):
    genotype = ind.genotype
    operation_indices = ind.operation_indices
    unique_ovens = ind.unique_ovens

    df_individual = pd.DataFrame(index=df_series.index, columns=df_ovens.index, data=0)

    for i, value in enumerate(genotype):
        operation = list(operation_indices.keys())[i % len(operation_indices)]
        oven = unique_ovens[i // len(operation_indices)]

        # Skip non-numeric values
        if not isinstance(df_individual.loc[oven, df_individual.columns[i % len(df_ovens)]], (int, float)):
            continue

        df_individual.loc[oven, df_individual.columns[i % len(df_ovens)]] = value

    fitness = 0

    for index, row in df_series.iterrows():
        temperature = row['temperature']
        operations = row.get('operations', [])

        for operation in operations:
            if df_individual.loc[index, df_individual.loc[index] > 0].any():
                oven_id = df_individual.loc[index].idxmax()
                # Convert oven_id to integer before using it as an index
                oven_id = int(oven_id)
                fitness += df_ovens.loc[oven_id, 'start_temp']

    return (fitness,)





def genetic_algorithm(df_ovens, df_series):
    creator.create("Fitness", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.Fitness)
    toolbox = base.Toolbox()
    toolbox.register("individual", individual_from_series_operations, df_series=df_series, df_ovens=df_ovens)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate, df_ovens=df_ovens, df_series=df_series)

    population = [toolbox.individual() for _ in range(50)]

    for ind in population:
        ind.fitness.values = toolbox.evaluate(ind, df_ovens=df_ovens, df_series=df_series)

    algorithms.eaMuPlusLambda(population, toolbox, mu=50, lambda_=200, cxpb=0.7, mutpb=0.2, ngen=100, stats=None, halloffame=None, verbose=True)

    best_individual = tools.selBest(population, k=1)[0]
    print("Лучшая индивидуальная оценка:", best_individual.fitness.values[0])


In [ ]:
for day, filename in enumerate(os.listdir(directory_path), start=0):
    if filename.endswith('.json'):
        file_path = os.path.join(directory_path, filename)
        day_data = read_json(file_path)
        df_ovens, df_series = analyze_day(day_data, day)
        list_df_ovens.append(df_ovens)
        list_df_series.append(df_series)

final_df_ovens = pd.concat(list_df_ovens, ignore_index=True)
final_df_series = pd.concat(list_df_series, ignore_index=True)

genetic_algorithm(final_df_ovens, final_df_series)

In [119]:
day_to_optimize = 0
file_path = os.path.join(directory_path, f'day-{day_to_optimize}.json')
day_data = read_json(file_path)
df_ovens, df_series = analyze_day(day_data, day_to_optimize)


genetic_algorithm(df_ovens, df_series)

Средняя начальная температура печей: 1092.7725118483413
Минимальная температура серий: 950


d:\PYTHONLANG\Python10.6\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
d:\PYTHONLANG\Python10.6\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	0     


ValueError: empty range for randrange() (1, 1, 0)

In [106]:
df_ovens

,start_temp,working_temps,operations,day
0,1220,"[960, 1030, 1190, 1100, 1230, 1070, 1110, 1080...","[prokat, kovka]",0
1,990,"[980, 1190, 1030, 1100, 990]","[otzhig, kovka]",0
2,1220,"[970, 1060, 1220]",[otzhig],0
3,1020,"[1060, 1100, 1140, 1240, 1150, 1020]",[kovka],0
4,1020,"[970, 1100, 1070, 980, 1180, 990, 1020]",[otzhig],0
...,...,...,...,...
417,1240,"[960, 1040, 1180, 1070, 1240]",[kovka],0
418,1050,"[1030, 1160, 1040, 980, 1110, 1020, 990, 1050]",[prokat],0
419,1140,"[1060, 1230, 1040, 1170, 990, 1140]",[kovka],0
420,950,"[1030, 1190, 1160, 1070, 980, 950]",[prokat],0


In [107]:
df_series

,temperature,operations,day
0,1080,"[{'name': 'nagrev', 'timing': 245}, {'name': '...",0
1,970,"[{'name': 'nagrev', 'timing': 62}, {'name': 'o...",0
2,1080,"[{'name': 'nagrev', 'timing': 76}, {'name': 'k...",0
3,980,"[{'name': 'nagrev', 'timing': 96}, {'name': 'o...",0
4,980,"[{'name': 'nagrev', 'timing': 82}, {'name': 'p...",0
...,...,...,...
2589,960,"[{'name': 'nagrev', 'timing': 281}, {'name': '...",0
2590,1060,"[{'name': 'nagrev', 'timing': 248}, {'name': '...",0
2591,1000,"[{'name': 'nagrev', 'timing': 182}, {'name': '...",0
2592,1090,"[{'name': 'nagrev', 'timing': 204}, {'name': '...",0
